# Analyzing Data

We will need to clean and format our data for training. By getting some statistics and visualization for our data we will do a better job of accounting for missing data and augmenting existing features for training.

In [ ]:
wine_utils = du.DataHelper()
wine_data = wine_utils.load_data()

In [ ]:
train_df = wine_data[0]
train_df

In [ ]:
for feature in train_df:
    total_entries =  len(train_df[feature])
    Num_Nan = train_df[feature].isnull().sum()
    print("Feature: {0} | Percent of NaN {1}".format(feature, float(Num_Nan)/float(total_entries)))

# Load Data and Embeddings

In [1]:
from lstm import RNNModel, Config, pad_sequences
import embeddings as emb
import tensorflow as tf
import data_utils as du
import pandas as pd

emb_helper = emb.embedding_helper(save_to_pickle = False, test_batch = 10000)

1873it [00:00, 9360.78it/s]

Loading Glove Embeddings:  /Users/LorenAC/AI/CS224N/FinalProjectRepo/Wine-NLP/embeddings/glove.42B.300d.txt


9537it [00:01, 9532.70it/s]

Done. 10002  words loaded!


In [2]:
data_helper = du.DataHelper(100)

X_train_df, X_dev_df = data_helper.X_train, data_helper.X_dev
label_helper_points = data_helper.labels_from_Y_cat("points")

# Get sub-embeddings for our given vocab

If we dont do this the embedding matrix may be too large too give to tensorflow graph

In [3]:
vocab, _ = data_helper.generate_vocab_and_word_frequencies() 
sub_emb_matrix, sub_tok2ind,sub_ind2tok, sub_unk_ind = emb_helper.get_sub_embeddings(vocab)

100%|██████████| 2404/2404 [00:00<00:00, 574012.68it/s]

Done. 1429  words loaded!


# Get our data ready for model preprocessing

This includes:

    1) encoding the words in our sentences as indices in the embedding matrix
    
    2) mapping labels into classes and getting into proper format
    
(here we use our label_helper which keeps track of the mapping between a label's value from dataset and its corresponding class number for training...this helper also stores the number of classes for the label which we will later pass to our model config)

In [4]:
import numpy as np
import time

X_train_tokens = X_train_df.as_matrix()
X_dev_tokens = X_dev_df.as_matrix()

X_train_indices = emb_helper.tok2ind_ind2tok(X_train_tokens, lookup_dict = sub_tok2ind, unk_indice = sub_unk_ind)
X_dev_indices = emb_helper.tok2ind_ind2tok(X_dev_tokens, lookup_dict = sub_tok2ind, unk_indice = sub_unk_ind)

train_raw = [X_train_indices, label_helper_points.train_classes]
dev_raw = [X_dev_indices, label_helper_points.dev_classes]

# Build and Run Model

This step includes:

    1) initializing our Config, Model

    2) preprocessing data further using model parameters

    3) opening our tensorflow Graph and Session

In [5]:
from util import write_conll, print_sentence

config = Config("lstm", n_classes = label_helper_points.num_classes, many2one = True)
embeddings = sub_emb_matrix
embeddings = np.asarray(embeddings)
config.embed_size = embeddings.shape[1]

with tf.Graph().as_default():
    print("Building model...",)
    start = time.time()
    model = RNNModel(data_helper, config, embeddings)
    print("took %.2f seconds", time.time() - start)
    
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    with tf.Session() as session:
        session.run(init)
        model.fit(session, saver, train_raw, dev_raw)
        output = model.output(session, dev_raw)
        sentences, class_labels, predictions = zip(*output)
        predictions = [[str(label_helper_points.class_2_lbl[cls]) for cls in preds] for preds in predictions]
        labels = [[str(label_helper_points.class_2_lbl[cls]) for cls in classes] for classes in class_labels]
        label_results = zip(labels, predictions)
        sentences = emb_helper.tok2ind_ind2tok(sentences, lookup_dict = sub_ind2tok, unk_indice = sub_unk_ind)
        output = zip(sentences, labels, predictions)
        
        #with open(model.config.conll_output, 'w') as f:
        #    write_conll(f, output)
        #with open(model.config.eval_output, 'w') as f:
        #    for sentence, label, prediction in output:
        #        print_sentence(f, sentence, label, prediction)
                

Building model...
Tensor("Mean_1:0", shape=(?,), dtype=int32)
took %.2f seconds 1.5493791103363037
Epoch %d out of %d 1 30
[2 0 5 1 0 3 2 0 3 4 4 3 1 1 5 6 2 3 0 0 1 1 0 2 0 0 1 1 1 1 1 1]
[2 0 4 2 1 0 4 1 2 4 3 1 1 1 1 0 3 2 1 1 0 0 4 2 5 3 4 0 0 0 2 1]
[0 0 4 0 4 0 5 1 4 0 1 1 3 0 0 0 0 1 0 2 1 1 4 0 8 1 2 3 0 1 8 1]
[4 1 0 0]
Loss:  2.70233

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [0]
true: [4]
pred: [0]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [0]
true: [5]
pred: [0]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [7]
pred: [0]
true: [8]
pred: [0]
true: [9]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [8]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [9]
pred: [0]
true: [4]
pre

[5 1 0 1 4 0 4 0 2 0 0 5 2 0 1 0 3 1 2 4 1 4 8 0 0 0 3 5 0 0 1 0]
[1 3 0 0 1 2 4 3 1 1 1 0 4 0 0 1 2 0 2 0 1 0 3 1 1 1 2 4 2 0 1 4]
[1 3 5 0 2 0 1 1 0 1 1 8 2 1 4 0 1 1 0 3 1 2 4 6 4 3 0 0 0 2 1 1]
[4 1 3 1]
Loss:  2.2917075

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [0]
true: [4]
pred: [0]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [0]
true: [5]
pred: [0]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [7]
pred: [0]
true: [8]
pred: [0]
true: [9]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [8]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [9]
pred: [0]
true: [4]
pred: [0]
true: [11]
pred: [0]
true: [4]
pred: [0]
true: [10]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [2]
p

[3 4 1 0 0 1 1 8 3 0 1 0 0 4 0 4 0 0 3 5 2 3 3 0 0 5 4 0 3 3 0 4]
[2 0 4 1 0 1 1 2 2 1 1 1 4 0 1 0 0 0 1 0 4 3 1 1 1 2 2 1 2 1 1 2]
[2 0 5 0 6 0 1 1 0 8 1 2 1 4 1 1 0 1 1 4 2 1 0 2 0 0 4 0 0 0 5 4]
[1 1 3 1]
Loss:  1.9710346

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [1]
true: [4]
pred: [1]
true: [5]
pred: [1]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [1]
true: [5]
pred: [0]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [1]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [7]
pred: [0]
true: [8]
pred: [1]
true: [9]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [8]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [0]
true: [11]
pred: [1]
true: [4]
pred: [0]
true: [10]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [2]
p

[4 3 4 0 3 1 0 5 0 3 0 1 2 0 2 0 1 0 0 1 1 1 0 1 2 0 2 4 1 4 1 1]
[0 3 1 2 1 1 1 4 0 1 3 4 2 0 4 4 4 5 2 8 0 0 0 0 0 2 0 4 1 0 1 2]
[1 1 5 3 5 2 3 4 2 1 2 4 0 0 1 1 1 0 8 0 1 3 1 1 0 0 1 1 0 0 3 6]
[1 0 1 0]
Loss:  1.8559321

4/4 [==============================] - 0s     
pred: [1]
true: [0]
pred: [1]
true: [1]
pred: [1]
true: [2]
pred: [1]
true: [3]
pred: [1]
true: [4]
pred: [1]
true: [5]
pred: [1]
true: [1]
pred: [1]
true: [0]
pred: [1]
true: [2]
pred: [1]
true: [5]
pred: [1]
true: [6]
pred: [1]
true: [4]
pred: [1]
true: [6]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [1]
true: [7]
pred: [1]
true: [8]
pred: [1]
true: [9]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [8]
pred: [1]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [1]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [1]
true: [11]
pred: [1]
true: [4]
pred: [1]
true: [10]
pred: [1]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [2]
p

[2 1 1 1 0 3 4 2 3 0 0 5 1 0 0 1 3 4 0 1 0 0 2 4 3 4 2 2 1 0 1 1]
[0 2 0 3 0 1 2 0 8 1 2 2 1 5 0 1 1 1 0 1 0 5 4 1 4 3 4 1 3 2 0 0]
[1 0 6 1 0 1 4 4 0 3 0 1 0 2 2 0 3 0 1 1 1 1 4 4 0 0 1 0 1 8 1 0]
[5 1 0 4]
Loss:  1.9798102

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [0]
true: [4]
pred: [0]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [0]
true: [5]
pred: [0]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [7]
pred: [0]
true: [8]
pred: [0]
true: [9]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [10]
pred: [0]
true: [5]
pred: [0]
true: [8]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [7]
pred: [0]
true: [9]
pred: [1]
true: [4]
pred: [0]
true: [11]
pred: [1]
true: [4]
pred: [0]
true: [10]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [0]
true: [2]
p

...

[5 5 3 1 3 0 1 4 4 0 0 1 1 1 0 1 1 4 1 1 8 0 3 1 0 1 1 1 0 0 0 4]
[0 1 3 0 0 0 0 5 1 0 6 1 4 2 1 4 0 3 0 1 0 3 2 0 1 3 0 0 3 4 2 1]
[0 4 1 1 0 2 2 1 1 2 8 1 0 2 5 3 2 4 2 2 4 0 0 2 0 1 1 1 1 4 0 1]
[2 0 0 4]
Loss:  1.3520466

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [1]
true: [4]
pred: [0]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [0]
true: [5]
pred: [3]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [0]
true: [7]
pred: [1]
true: [8]
pred: [3]
true: [9]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [8]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [3]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [1]
true: [11]
pred: [1]
true: [4]
pred: [1]
true: [10]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [2]
p

[2 2 8 1 1 3 1 1 4 0 0 4 0 0 3 1 3 0 3 0 0 0 1 4 0 1 8 4 1 1 0 3]
[2 2 0 0 1 2 1 5 0 0 2 0 4 1 0 6 3 2 1 0 0 4 3 0 5 2 0 0 0 3 2 4]
[0 1 1 0 1 4 1 4 1 1 2 1 1 0 2 5 1 0 1 4 1 0 4 0 1 1 1 1 2 4 0 5]
[0 1 1 3]
Loss:  1.1933709

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [1]
true: [4]
pred: [0]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [0]
true: [5]
pred: [0]
true: [6]
pred: [0]
true: [4]
pred: [0]
true: [6]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [0]
true: [7]
pred: [1]
true: [8]
pred: [3]
true: [9]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [8]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [3]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [1]
true: [11]
pred: [1]
true: [4]
pred: [1]
true: [10]
pred: [0]
true: [0]
pred: [0]
true: [10]
pred: [1]
true: [2]
p

[0 0 0 5 0 1 2 1 6 3 1 3 3 4 5 5 1 1 2 1 4 2 2 0 1 0 0 0 0 8 8 0]
[1 4 3 0 1 1 0 0 4 1 1 0 2 4 3 0 2 1 1 1 1 4 1 4 2 1 1 3 2 2 1 0]
[0 3 0 4 1 4 0 0 4 1 2 0 0 3 0 1 0 4 0 0 0 2 2 5 1 1 3 0 0 1 0 1]
[1 4 1 1]
Loss:  1.2038987

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [1]
true: [4]
pred: [3]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [3]
true: [5]
pred: [3]
true: [6]
pred: [0]
true: [4]
pred: [1]
true: [6]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [3]
true: [7]
pred: [1]
true: [8]
pred: [3]
true: [9]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [8]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [3]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [1]
true: [11]
pred: [1]
true: [4]
pred: [1]
true: [10]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [2]
p

Epoch %d out of %d 29 30
[1 0 1 0 2 0 4 3 0 2 2 1 1 1 3 0 1 0 3 2 1 1 1 0 1 1 4 8 1 2 0 1]
[0 0 1 4 0 4 0 1 4 4 1 0 0 0 1 2 2 3 4 2 2 0 3 4 0 8 0 1 1 6 0 4]
[0 0 5 0 0 1 1 0 0 4 1 0 3 2 0 1 3 1 3 1 5 4 1 2 0 0 4 1 5 2 0 1]
[5 1 3 1]
Loss:  1.1891707

4/4 [==============================] - 0s     
pred: [0]
true: [0]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [0]
true: [3]
pred: [1]
true: [4]
pred: [3]
true: [5]
pred: [0]
true: [1]
pred: [0]
true: [0]
pred: [0]
true: [2]
pred: [3]
true: [5]
pred: [3]
true: [6]
pred: [0]
true: [4]
pred: [1]
true: [6]
pred: [1]
true: [2]
pred: [1]
true: [7]
pred: [3]
true: [7]
pred: [1]
true: [8]
pred: [3]
true: [9]
pred: [4]
true: [10]
pred: [1]
true: [5]
pred: [0]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [8]
pred: [0]
true: [0]
pred: [1]
true: [10]
pred: [0]
true: [1]
pred: [0]
true: [2]
pred: [3]
true: [7]
pred: [1]
true: [9]
pred: [1]
true: [4]
pred: [4]
true: [11]
pred: [1]
true: [4]
pred: [4]
true: [10]
pred: [0]
true: [0]
pred: [1]
true: [

4/4 [==============================] - 0s     
pred: [1]
true: [1]
pred: [1]
true: [4]
pred: [1]
true: [3]
pred: [1]
true: [1]
pred: [1]
true: [2]
pred: [1]
true: [0]
pred: [1]
true: [10]
pred: [1]
true: [5]
pred: [1]
true: [12]
pred: [1]
true: [2]
pred: [3]
true: [5]
pred: [1]
true: [2]
pred: [0]
true: [3]
pred: [0]
true: [0]
pred: [1]
true: [3]
pred: [1]
true: [0]
pred: [1]
true: [4]
pred: [0]
true: [7]
pred: [0]
true: [7]
pred: [1]
true: [1]
pred: [1]
true: [9]
pred: [1]
true: [10]
pred: [3]
true: [10]
pred: [1]
true: [2]
pred: [0]
true: [0]
pred: [0]
true: [0]
pred: [1]
true: [7]
pred: [1]
true: [2]
pred: [1]
true: [12]
pred: [1]
true: [0]
pred: [1]
true: [7]
pred: [0]
true: [2]
pred: [1]
true: [3]
pred: [0]
true: [6]
pred: [1]
true: [0]
pred: [1]
true: [13]
pred: [1]
true: [4]
pred: [0]
true: [4]
pred: [1]
true: [1]
pred: [1]
true: [8]
pred: [1]
true: [1]
pred: [1]
true: [9]
pred: [0]
true: [7]
pred: [3]
true: [10]
pred: [1]
true: [1]
pred: [1]
true: [13]
pred: [1]
true: [6]
pred:

In [ ]:
dev_raw

# Define General Model for Running in TensorFlow

In [ ]:
def do_evaluate(args):
    config = Config(args)
    helper = ModelHelper.load(args.model_path)
    input_data = read_conll(args.data)
    embeddings = load_embeddings(args, helper)
    config.embed_size = embeddings.shape[1]

    with tf.Graph().as_default():
        logger.info("Building model...",)
        start = time.time()
        model = RNNModel(helper, config, embeddings)

        logger.info("took %.2f seconds", time.time() - start)

        init = tf.global_variables_initializer()
        saver = tf.train.Saver()

        with tf.Session() as session:
            session.run(init)
            saver.restore(session, model.config.model_output)
            for sentence, labels, predictions in model.output(session, input_data):
                predictions = [LBLS[l] for l in predictions]
                print_sentence(args.output, sentence, labels, predictions)

In [ ]:
x = [[3], [5], [5]]
y = [5]
x = np.asarray(x)
x = x + [y]*4
#np.expand_dims(x, 2).shape
x

In [ ]:
X_train

In [13]:
x = [86, 1]
np.expand_dims(x,1)

array([[86],
       [ 1]])

In [119]:
import numpy as np
ex = np.array([[[0, 1, 2], [1, 5, 6], [2, 3, 2]], [[3, 3, 4], [4, 4, 3], [5, 1, 2]]])
labels = np.array([[0], [3]])
mask = np.array([[True, True, False], [True, True, False]])

In [120]:
print(ex.shape, labels.shape)

(2, 3, 3) (2, 1)


In [121]:
ex_masked = ex[mask]
#lab_masked = labels[mask]

In [124]:
print(ex, ex_masked)

[[[0 1 2]
  [1 5 6]
  [2 3 2]]

 [[3 3 4]
  [4 4 3]
  [5 1 2]]] [[0 1 2]
 [1 5 6]
 [3 3 4]
 [4 4 3]]


In [125]:
print(ex_masked.shape, labels.shape)

(4, 3) (2, 1)


In [118]:
y = y==0

In [67]:
y

array([[ True,  True,  True,  True,  True,  True,  True, False, False,
        False],
       [ True,  True,  True,  True,  True,  True,  True, False, False,
        False],
       [ True,  True,  True,  True,  True,  True,  True, False, False,
        False]])

In [68]:
x = x[y]

In [69]:
x.shape

(21, 5)

In [70]:
np.reshape(x, (3, -1, 5))

array([[[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [0., 0., 0., 0., 0.]],

       [[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [0., 0., 0., 0., 0.]],

       [[3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [3., 3., 3., 3., 3.],
        [0., 0., 0., 0., 0.]]])